# Data Cleaning

In [5]:
import pandas as pd
import pickle
import numpy as np

## National Survey of Family Growth (NSFG)
The NSFG was conducted by the National Center for Health Statistics (NCHS), which is affiliated with the Center for Diseas Control and Prevention (CDC). 

This data set contains a plethora of data on family growth. Specifically, I'm examining the pregnancy data that they've obtained. The data I'm using come from 2015-2017 and are based on survey results obtained from 9,553 different people who were pregnant. The data contain information about race, miscarriage, smoking habits, religion, age, and many other factors.

In [6]:
# Check my datacollection folder to get all the data files I downloaded there using my scraper
with open('../datacollection/all_data_files.txt', 'r') as rdfs:
    raw_data_files = [x.strip() for x in rdfs.readlines()]
raw_data_files

['2015_2017_FemPregData.dat',
 '2015_2017_FemRespData.dat',
 '2015_2017_MaleData.dat',
 'stata/2015_2017_FemPregSetup.dct',
 'stata/2015_2017_FemRespSetup.dct',
 'stata/2015_2017_MaleSetup.dct']

In [7]:
# Get the stata dicts for reading nsfg files
stata_dicts = []
rdfs2 = []
for rdf in raw_data_files:
    if rdf.endswith('.dct'):
        stata_dicts.append(rdf)
    else:
        rdfs2.append(rdf)
        
# Get rid of stata dicts from data files
raw_data_files = rdfs2
stata_dicts

['stata/2015_2017_FemPregSetup.dct',
 'stata/2015_2017_FemRespSetup.dct',
 'stata/2015_2017_MaleSetup.dct']

Since the NSFG files are in a weird format, we will begin by converting them to .csv files so that pandas can easily read them.

In [8]:
def create_csv(columns, column_range, data_file):
    """Create csv from NSFG data file."""
    
    top_line = ','.join(columns)
    data_csv = [top_line+'\n']
    
    with open('../datacollection/'+data_file,'r') as data:
        data_raw = data.readlines()

    for line in data_raw:
        j = 0
        for idx in column_range[1:-1]:
            line = line[:idx+j] + ',' + line[idx+j:]
            j += 1 # Increment j since we just added another character
        data_csv.append(line)
    
    # Write to the file
    with open('../data/'+data_file.replace('.dat','.csv'),'w') as data:
        data.writelines(data_csv)    

In [9]:
for data_file in raw_data_files:
    if data_file[-4:] == '.dat': # We only care about nsfg files
        dct_file = None
        
        # Find the dict applying to this data file
        for dct in stata_dicts:
            if data_file[:-8] in dct:
                dct_file = dct
                break
        if dct_file is None:
            raise ValueError("Could not find data dictionary")
        
        # Use data_dct to save the data file as csv
        with open('../datacollection/'+dct_file,'r') as dct:
            raw_dct = dct.readlines()
        
        # Drop the header
        raw_dct = ''.join(raw_dct[14:-1])
            
        raw_dct = [
            [x for x in line.split(' ') if x != ''] 
            for line in raw_dct.split('\n')
        ]
        
        # Get the column names/descriptions
        column_range = [int(line[0][8:-1])-1 for line in raw_dct[:-1]]
        column_names = [line[2] for line in raw_dct[:-1]]
        column_descriptions = [' '.join(line[4:]) for line in raw_dct[:-1]]
        column_info_dict = {x: y for x,y in zip(column_names, column_descriptions)}
        
        # Save the column info so we can query it later
        with open(data_file.replace('.dat','.pickle'), 'wb') as handle:
            pickle.dump(
                column_info_dict, 
                handle, 
                protocol=pickle.HIGHEST_PROTOCOL
            )

        create_csv(column_names, column_range, data_file)        

## NSFG Cleaning
Now that the data is in a readable format (csv), we will extract it and clean up whatever columns we can, drop unnneeded columns, etc.

In [10]:
# Read to a dataframe
df = pd.read_csv('../data/2015_2017_FemPregData.csv',low_memory=False)
df.head()

,CASEID,PREGORDR,HOWPREG_N,HOWPREG_P,MOSCURRP,NOWPRGDK,PREGEND1,PREGEND2,HOWENDDK,NBRNALIV,...,SECU,SEST,CMINTVW,CMLSTYR,CMJAN3YR,CMJAN4YR,CMJAN5YR,QUARTER,PHASE,INTVWYEAR
0,70627,1,,,,,6,,,1,...,3,322,1394,1382,1357,1345,1333,18,12016,NaN
1,70627,2,,,,,1,,,,...,3,322,1394,1382,1357,1345,1333,18,12016,NaN
2,70627,3,,,,,6,,,1,...,3,322,1394,1382,1357,1345,1333,18,12016,NaN
3,70628,1,,,,,6,,,1,...,2,366,1409,1397,1369,1357,1345,23,12017,NaN
4,70628,2,,,,,6,,,1,...,2,366,1409,1397,1369,1357,1345,23,12017,NaN


In [11]:
# Get descriptions of each column
with open('2015_2017_FemPregData.pickle', 'rb') as fs:
    cols = pickle.load(fs)
print('Number of columns:', len(cols))

Number of columns: 248


Right now, all of the unrecorded values are whitespace. We want these to be consistently recorded as NaNs, so we can tell what data we're missing.

In [12]:
df.replace({' ': np.nan, '  ': np.nan, '   ': np.nan, '    ': np.nan}, inplace=True)
df.head()

,CASEID,PREGORDR,HOWPREG_N,HOWPREG_P,MOSCURRP,NOWPRGDK,PREGEND1,PREGEND2,HOWENDDK,NBRNALIV,...,SECU,SEST,CMINTVW,CMLSTYR,CMJAN3YR,CMJAN4YR,CMJAN5YR,QUARTER,PHASE,INTVWYEAR
0,70627,1,NaN,NaN,NaN,NaN,6,NaN,NaN,1,...,3,322,1394,1382,1357,1345,1333,18,12016,NaN
1,70627,2,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,3,322,1394,1382,1357,1345,1333,18,12016,NaN
2,70627,3,NaN,NaN,NaN,NaN,6,NaN,NaN,1,...,3,322,1394,1382,1357,1345,1333,18,12016,NaN
3,70628,1,NaN,NaN,NaN,NaN,6,NaN,NaN,1,...,2,366,1409,1397,1369,1357,1345,23,12017,NaN
4,70628,2,NaN,NaN,NaN,NaN,6,NaN,NaN,1,...,2,366,1409,1397,1369,1357,1345,23,12017,NaN


If most of the data in a column is NaN, then we will drop that column. The threshold we will use is 60%.

In [13]:
df = df.loc[:,(df.isnull().mean() < 0.60)]
df.head()

,CASEID,PREGORDR,PREGEND1,NBRNALIV,BORNALIV,GESTASUN_M,GESTASUN_W,WKSGEST,MOSGEST,BABYSEX1,...,WGT2015_2017,SECU,SEST,CMINTVW,CMLSTYR,CMJAN3YR,CMJAN4YR,CMJAN5YR,QUARTER,PHASE
0,70627,1,6,1,1,0,40,40,9,2,...,19877.457610,3,322,1394,1382,1357,1345,1333,18,12016
1,70627,2,1,NaN,NaN,0,14,14,3,NaN,...,19877.457610,3,322,1394,1382,1357,1345,1333,18,12016
2,70627,3,6,1,1,0,39,39,9,1,...,19877.457610,3,322,1394,1382,1357,1345,1333,18,12016
3,70628,1,6,1,1,9,0,39,9,2,...,4221.017695,2,366,1409,1397,1369,1357,1345,23,12017
4,70628,2,6,1,1,9,0,39,9,2,...,4221.017695,2,366,1409,1397,1369,1357,1345,23,12017


Now that we are down to 122 columns, there are quite a few more that we can consolidate or drop. For instance, the columns 'CMJAN3YR', 'CMJAN4YR', etc. are numbers that record the 'Century month of January 4 years prior to the year of the interview'. This doesn't seem helpful. After looking through descriptions on a number of the columns, I've decided to drop all the columns in the code below.

In [14]:
df.drop(columns=['CMJAN5YR',
                 'CMJAN4YR',
                 'CMJAN3YR',
                 'CMINTVW',
                 'CMLSTYR',
                 'PREGEND1',
                 'NBRNALIV',
                ],inplace=True)
df.head()

,CASEID,PREGORDR,BORNALIV,GESTASUN_M,GESTASUN_W,WKSGEST,MOSGEST,BABYSEX1,BIRTHWGT_LB1,BIRTHWGT_OZ1,...,PUBASSIS_I,POVERTY_I,LABORFOR_I,RELIGION_I,METRO_I,WGT2015_2017,SECU,SEST,QUARTER,PHASE
0,70627,1,1,0,40,40,9,2,7,8,...,0,0,0,0,0,19877.457610,3,322,18,12016
1,70627,2,NaN,0,14,14,3,NaN,NaN,NaN,...,0,0,0,0,0,19877.457610,3,322,18,12016
2,70627,3,1,0,39,39,9,1,9,2,...,0,0,0,0,0,19877.457610,3,322,18,12016
3,70628,1,1,9,0,39,9,2,6,9,...,0,0,0,0,0,4221.017695,2,366,23,12017
4,70628,2,1,9,0,39,9,2,7,0,...,0,0,0,0,0,4221.017695,2,366,23,12017


A few of our columns (each of the ones ending in '\_I') are imputation flags for another column, meaning that the NCHS recorded whether that variable was filled out in the questionnaire, or imputed using either 'Multiple Regression Imputation' or 'Logical Imputation'. We will drop these columns, under the assumption that the government can be trusted to properly impute these data.

In [15]:
for col in df.columns:
    if col[-2:] == '_I':
        df.drop(columns=[col],inplace=True)
df.head()

,CASEID,PREGORDR,BORNALIV,GESTASUN_M,GESTASUN_W,WKSGEST,MOSGEST,BABYSEX1,BIRTHWGT_LB1,BIRTHWGT_OZ1,...,POVERTY,LABORFOR,RELIGION,METRO,BRNOUT,WGT2015_2017,SECU,SEST,QUARTER,PHASE
0,70627,1,1,0,40,40,9,2,7,8,...,500,1,2,1,5,19877.457610,3,322,18,12016
1,70627,2,NaN,0,14,14,3,NaN,NaN,NaN,...,500,1,2,1,5,19877.457610,3,322,18,12016
2,70627,3,1,0,39,39,9,1,9,2,...,500,1,2,1,5,19877.457610,3,322,18,12016
3,70628,1,1,9,0,39,9,2,6,9,...,189,1,3,1,5,4221.017695,2,366,23,12017
4,70628,2,1,9,0,39,9,2,7,0,...,189,1,3,1,5,4221.017695,2,366,23,12017


I want to choose more columns to drop. In order to do so, I will examine each of the remaining column descriptions, look them up if I need more information, and then decide which other columns to drop.

In [16]:
for col in df.columns:
    #print(col,':',cols[col])
    pass
to_drop = ['GESTASUN_M','WKSGEST','MOSGEST','KIDAGE','HPAGELB', 
           'ANYUSINT','NBRNLV_S','TIMOKHP', 'WANTRESP',
           'OUTCOM_S','BIRTHORD','AGEPREG', 'DATEND','DATECON',
           'FMARCON5','LBW1','LIVCHILD','BFEEDWKS','OLDWANTR',
           'OLDWANTP','AGER','AGESCRN','RCURPREG','HISPRACE',
           'HISPRACE2','PARITY','BRNOUT','SECU','SEST','QUARTER',
           'PHASE', 'BABYSEX1', 'GESTASUN_W', 'PMARPREG',
           'BABYDOB_Y', 'ANYNURSE1', 'WTHPART1', 'WTHPART2', 
           'TIMINGOK', 'RMARCON6', 'RMAROUT6', 'FMAROUT5', 
           'FMARITAL', 'HIEDUC', 'CURR_INS', 'PRGOUTCOME'
          ]
df.drop(columns=to_drop,inplace=True)
df.head()

,CASEID,PREGORDR,BORNALIV,BIRTHWGT_LB1,BIRTHWGT_OZ1,EVUSEINT,STOPDUSE,WANTBOLD,HPWNOLD,COHPBEG,...,EDUCAT,RACE,HISPANIC,PREGNUM,PUBASSIS,POVERTY,LABORFOR,RELIGION,METRO,WGT2015_2017
0,70627,1,1,7,8,1,1,NaN,1,NaN,...,16,2,2,3,2,500,1,2,1,19877.457610
1,70627,2,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,...,16,2,2,3,2,500,1,2,1,19877.457610
2,70627,3,1,9,2,NaN,NaN,NaN,1,NaN,...,16,2,2,3,2,500,1,2,1,19877.457610
3,70628,1,1,6,9,1,5,5,6,5,...,15,1,2,3,2,189,1,3,1,4221.017695
4,70628,2,1,7,0,1,5,1,6,1,...,15,1,2,3,2,189,1,3,1,4221.017695


In [20]:
for col in df.columns:
    print(col,':', cols[col])

CASEID : "Case identification number"
PREGORDR : "Pregnancy order (number)"
BORNALIV : "Number of babies born alive from this pregnancy"
BIRTHWGT_LB1 : "BD-3 Birthweight in Pounds - 1st baby from this pregnancy"
BIRTHWGT_OZ1 : "BD-3 Birthweight in Ounces - 1st baby from this pregnancy"
EVUSEINT : "EG-1 Use any method in pregnancy interval?"
STOPDUSE : "EG-2 Before you became preg, stop using all methods?"
WANTBOLD : "EG-6 Right bef preg, want to have baby at any time in future?"
HPWNOLD : "EG-16 Right bef preg, did the father want R to have baby at any time in future?"
COHPBEG : "EG-18a Was R living w/father of preg at beginning of preg"
COHPEND : "EG-18b Was R living w/father of preg when preg ended/baby was born"
PRGLNGTH : "Duration of completed pregnancy in weeks"
OUTCOME : "Pregnancy outcome"
AGECON : "Age at time of conception"
WANTPART : "Wantedness of pregnancy -- R's partner (RECODE)"
NEWWANTR : "Detailed wantedness of pregnancy - respondent"
RMARITAL : "Informal Marital Statu

The problem now is that much of our data consists of answers to survey questions that are recorded as numbers. We will need to go through each of these questions and convert the numbers to categories (strings). The information as to how these were encoded can be found at the following website: https://www.icpsr.umich.edu/icpsradmin/nsfg/variableGroupParent/14242?studyNumber=10001

In [17]:
# Specifically, OUTCOME lets us know about miscarriages vs other pregnancy outcomes.
df['OUTCOME'] = pd.to_numeric(df['OUTCOME'])
df['OUTCOME'].replace({
    1: 'LIVE_BIRTH',
    2: 'ABORTION',
    3: 'STILLBIRTH',
    4: 'MISCARRIAGE',
    5: 'ECTOPIC',
    6: 'CURRENT' # Meaning not born yet
    },
    inplace=True
)
# Get rid of current pregnancies
df = df[df['OUTCOME'] != 'CURRENT']
df['OUTCOME']

0        LIVE_BIRTH
1       MISCARRIAGE
2        LIVE_BIRTH
3        LIVE_BIRTH
4        LIVE_BIRTH
           ...     
9547     LIVE_BIRTH
9548     LIVE_BIRTH
9549        ECTOPIC
9550     LIVE_BIRTH
9551     LIVE_BIRTH
Name: OUTCOME, Length: 9352, dtype: object

The 'BORNALIV' column records the number of children born alive from the pregnancy being recorded. However, if a live birth did not occur, rather than being recorded as 0, it is recorded as NaN.

In [14]:
df['BORNALIV'].fillna(0,inplace=True)

In the following block of code, we encode a whole lot of survey data and convert to several numeric types.

In [15]:
df['PREGORDR'] = pd.to_numeric(df['PREGORDR'])
df['BORNALIV'] = pd.to_numeric(df['BORNALIV'])
df['EVUSEINT'] = pd.to_numeric(df['EVUSEINT'])
df['EVUSEINT'].replace({
    1: 'Yes',
    5: 'No',
    8: 'Refused',
    9: 'Dont-know',
},inplace=True)
df['STOPDUSE'] = pd.to_numeric(df['STOPDUSE'])
df['STOPDUSE'].replace({
    1: 'Yes',
    5: 'No',
    8: np.nan,
    9: np.nan,
},inplace=True)
df['WANTBOLD'] = pd.to_numeric(df['WANTBOLD'])
df['WANTBOLD'].replace({
    1: 'Yes',
    5: 'No',
    6: np.nan,
    8: np.nan,
    9: np.nan
},inplace=True)
df['HPWNOLD'] = pd.to_numeric(df['HPWNOLD'])
df['HPWNOLD'].replace({
    1: 'Yes',
    5: 'No',
    6: 'Dont-know',
    7: np.nan,
    8: np.nan,
    9: np.nan,
},inplace=True)
df['COHPBEG'] = pd.to_numeric(df['COHPBEG'])
df['COHPBEG'].replace({
    1: 'Yes',
    5: 'No',
    8: np.nan,
    9: np.nan
},inplace=True)
df['COHPEND'] = pd.to_numeric(df['COHPEND'])
df['COHPEND'].replace({
    1: 'Yes',
    5: 'No',
    8: np.nan,
    9: np.nan
},inplace=True)
df['PRGLNGTH'] = pd.to_numeric(df['PRGLNGTH']) # FIXME: needs checks
df['AGECON'] = pd.to_numeric(df['AGECON'])
df['WANTPART'] = pd.to_numeric(df['WANTPART'])
df['WANTPART'].replace({
    1: 'Later-overdue',
    2: 'Right-time',
    3: 'Too-soon',
    4: 'Didnt-care',
    5: 'Unwanted',
    6: 'Not-sure'
},inplace=True)
df['NEWWANTR'] = pd.to_numeric(df['NEWWANTR'])
df['NEWWANTR'].replace({
    1: 'Later-overdue',
    2: 'Right-time',
    3: 'Too-soon<2-years',
    4: 'Too-soon>=2-years',
    5: 'Didnt-care',
    6: 'Unwanted',
    7: 'Not-sure'
},inplace=True)
df['RMARITAL'] = pd.to_numeric(df['RMARITAL'])
df['RMARITAL'].replace({
    1: 'Married',
    2: 'Cohabiting',
    3: 'Widowed',
    4: 'Divorced',
    5: 'Separated',
    6: 'Never-Married'
},inplace=True)
df['EDUCAT'] = pd.to_numeric(df['EDUCAT'])
df['RACE'] = pd.to_numeric(df['RACE'])
df['RACE'].replace({
    1: 'Black',
    2: 'White',
    3: 'Other'
},inplace=True)
df['HISPANIC'] = pd.to_numeric(df['HISPANIC'])
df['HISPANIC'].replace({
    1: 1,
    2: 0
},inplace=True)
df['PREGNUM'] = pd.to_numeric(df['PREGNUM'])
df['PUBASSIS'] = pd.to_numeric(df['PUBASSIS'])
df['PUBASSIS'].replace({
    1: 1,
    2: 0
},inplace=True)
df['POVERTY'] = pd.to_numeric(df['POVERTY'])/100 # Percentage
df['LABORFOR'] = pd.to_numeric(df['LABORFOR'])
df['LABORFOR'].replace({
    1: 'Full-time',
    2: 'Part-time',
    3: 'Temp-or-ill',
    4: 'Maternity-Family-leave',
    5: 'Looking-for-work',
    6: 'School',
    7: 'Stay-at-home',
    8: 'Caring-for-family',
    9: 'Other'
},inplace=True)
df['RELIGION'] = pd.to_numeric(df['RELIGION'])
df['RELIGION'].replace({
    1: 'None',
    2: 'Catholic',
    3: 'Protestant',
    4: 'Other'
},inplace=True)
df['METRO'] = pd.to_numeric(df['METRO'])
df['METRO'].replace({
    1: 1,
    2: 1,
    3: 0
},inplace=True)
df['WGT2015_2017'] = pd.to_numeric(df['WGT2015_2017'])

# Potential Problems

With that, our data is pretty clean! However, there are a few potential problems. There are still some missing values that could cause us problems. Another potential problem is that a few numeric values don't make sense: for instance, the length of pregnancy has values as high as 49 weeks. This seems unlikely (I'm not a medical professional, but I think the optimal time is around 40 weeks). Upon further examination, only a single data point has a value higher than 44 weeks (at 49 weeks). I will consider this to be an outlier that could be possible, and not remove it from the dataset.

# Suitability of Data
Originally, I wanted to look at medical data to answer some of the questions in my proposal. However, finding good medical data is very difficult. This dataset is a really good one, and I think some great information can be gleaned from it.

The following quotation from the CDC website (https://www.cdc.gov/nchs/nsfg/) explains what this dataset is for: 

    The National Survey of Family Growth (NSFG) gathers information on family life, marriage and divorce, pregnancy, infertility, use of contraception, and men’s and women’s health. The survey results are used by the U.S. Department of Health and Human Services and others to plan health services and health education programs, and to do statistical studies of families, fertility, and health.
    
This dataset does have some problems if we want to extrapolate its findings to the general public. The surveyors deliberately over-sampled certain populations (specifically minorities such as blacks and teenagers). This is because they were worried that they wouldn't have enough predictive power for these minorities if they surveyed them 'to scale'. In order to increase their precision for these subgroups of the population, they over-sampled these groups. This could cause problems in our statistical analysis (if we hope to extrapolate the findings to the general U.S. population). In order to account for this oversampling of minorities (and hence undersampling of non-minorities), the dataset includes weights that can be used to generalize the data to the total U.S. population. The column 'WGT2015_2017' contains the weights that should be used for each sample. The NSFG explains the weights in the following way:

    For purposes of description, it may be useful to observe that the final weight can be interpreted as the number of persons in the population that an individual NSFG respondent represents. A final weight for a teenage Hispanic female of 2,000 means that this sample respondent represents herself and 1,999 other similar women in the population. The NSFG 2015-2017 final weights are values greater than 1, and when summed across a subgroup or the total sample are expected to provide an estimate of the total number of persons in that subgroup in the U.S. household population.

 For more information on weights and how they were calculated, one can review the survey's documentation at https://www.cdc.gov/nchs/data/nsfg/PUF3-NSFG-2015-2017-Weighting-Design_02Oct2019.pdf.
 

Another potential problem with the suitability of this data stems from the fact that it is a survey. People sometimes misrepresent the truth on surveys. This might be to appear better or it might be because the person's recollection isn't true. Sometimes these survey questions asked women about a pregnancy that was a few years in the past, which could have led to the woman forgetting certain details or giving details that were incorrect. Additionally, the survey asked if a pregnancy ended in abortion. Since abortion is of questionable legality depending on the state and can be stigmatized, it's possible (and in my opinion, likely) that the number of abortions is underrepresented. Also, some instances of reported miscarriage may have actually been abortions.

Despite some of the problems with the dataset, this dataset is a very good one. The NSFG specifically traveled throughout the entire U.S., sampling individuals from every state. They further subdivided states into county-based areas, each of which was visited and sampled. This means that the survey should have good predictive power for the entire U.S. (although extrapolating internationally could be unwise).

# Other Revisions
With this data, I am not confident that I can predict miscarriages (I think that medical data would be more useful for that). However, I can identify potential risks and groups of people who might be more at risk for miscarriage. This could help doctors and public health professionals to know who they should focus on helping. It may also shed light on where research should be done to determine the causes of miscarriages.

In [16]:
# Save the cleaned data
df.to_csv('cleaned_nsfg_data.csv')